# [7] Amazon Fine Food Reviews Analysis


Data Source: https://www.kaggle.com/snap/amazon-fine-food-reviews

The Amazon Fine Food Reviews dataset consists of reviews of fine foods from Amazon.<br>

Number of reviews: 568,454<br>
Number of users: 256,059<br>
Number of products: 74,258<br>
Timespan: Oct 1999 - Oct 2012<br>
Number of Attributes/Columns in data: 10 

Attribute Information:

1. Id
2. ProductId - unique identifier for the product
3. UserId - unqiue identifier for the user
4. ProfileName
5. HelpfulnessNumerator - number of users who found the review helpful
6. HelpfulnessDenominator - number of users who indicated whether they found the review helpful or not
7. Score - rating between 1 and 5
8. Time - timestamp for the review
9. Summary - brief summary of the review
10. Text - text of the review


#### Objective:
Given a review, determine whether the review is positive (Rating of 4 or 5) or negative (rating of 1 or 2).

<br>
[Q] How to determine if a review is positive or negative?<br>
<br> 
[Ans] We could use the Score/Rating. A rating of 4 or 5 could be cosnidered a positive review. A review of 1 or 2 could be considered negative. A review of 3 is nuetral and ignored. This is an approximate and proxy way of determining the polarity (positivity/negativity) of a review.




## [7.1] Loading the data

The dataset is available in two forms
1. .csv file
2. SQLite Database

In order to load the data, We have used the SQLITE dataset as it easier to query the data and visualise the data efficiently.
<br> 

Here as we only want to get the global sentiment of the recommendations (positive or negative), we will purposefully ignore all Scores equal to 3. If the score id above 3, then the recommendation wil be set to "positive". Otherwise, it will be set to "negative".

In [3]:
%matplotlib inline

import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import cross_val_score
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer



# using the SQLite Table to read data.
con = sqlite3.connect('database.sqlite') 



#filtering only positive and negative reviews i.e. 
# not taking into consideration those reviews with Score=3
filtered_data = pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3
""", con) 




# Give reviews with Score>3 a positive rating, and reviews with a score<3 a negative rating.
def partition(x):
    if x < 3:
        return 'negative'
    return 'positive'

#changing reviews with score less than 3 to be positive and vice-versa
actualScore = filtered_data['Score']
positiveNegative = actualScore.map(partition) 
filtered_data['Score'] = positiveNegative

In [17]:
filtered_data.shape #looking at the number of attributes and size of the data
filtered_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,negative,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,positive,1350777600,Great taffy,Great taffy at a great price. There was a wid...


#  Exploratory Data Analysis

## [7.1.2] Data Cleaning: Deduplication

It is observed (as shown in the table below) that the reviews data had many duplicate entries. Hence it was necessary to remove duplicates in order to get unbiased results for the analysis of the data.  Following is an example:

In [2]:
display= pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3 AND UserId="AR5J8UI46CURR"
ORDER BY ProductID
""", con)
display

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
1,138317,B000HDOPYC,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
2,138277,B000HDOPYM,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
3,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
4,155049,B000PAQ75C,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


As can be seen above the same user has multiple reviews of the with the same values for HelpfulnessNumerator, HelpfulnessDenominator, Score, Time, Summary and Text  and on doing analysis it was found that <br>
<br> 
ProductId=B000HDOPZG was Loacker Quadratini Vanilla Wafer Cookies, 8.82-Ounce Packages (Pack of 8)<br>
<br> 
ProductId=B000HDL1RQ was Loacker Quadratini Lemon Wafer Cookies, 8.82-Ounce Packages (Pack of 8) and so on<br>

It was inferred after analysis that reviews with same parameters other than ProductId belonged to the same product just having different flavour or quantity. Hence in order to reduce redundancy it was decided to eliminate the rows having same parameters.<br>

The method used for the same was that we first sort the data according to ProductId and then just keep the first similar product review and delelte the others. for eg. in the above just the review for ProductId=B000HDL1RQ remains. This method ensures that there is only one representative for each product and deduplication without sorting would lead to possibility of different representatives still existing for the same product.

In [60]:
#Sorting data according to ProductId in ascending order
sorted_data=filtered_data.sort_values('Time', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

In [61]:
#Deduplication of entries
final=sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"}, keep='first', inplace=False)
final.shape

(364173, 10)

In [5]:
#Checking to see how much % of data still remains
(final['Id'].size*1.0)/(filtered_data['Id'].size*1.0)*100

69.25890143662969

<b>Observation:-</b> It was also seen that in two rows given below the value of HelpfulnessNumerator is greater than HelpfulnessDenominator which is not practically possible hence these two rows too are removed from calcualtions

In [6]:
display= pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3 AND Id=44737 OR Id=64422
ORDER BY ProductID
""", con)
display


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,5,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...
1,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,4,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...


In [62]:
final=final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]
#Y = final['labels']

In [5]:
#Before starting the next phase of preprocessing lets see the number of entries left
print(final.shape)

#How many positive and negative reviews are present in our dataset?
final['Score'].value_counts()

(364171, 10)


positive    307061
negative     57110
Name: Score, dtype: int64

## 7.2.3  Text Preprocessing: Stemming, stop-word removal and Lemmatization.

Now that we have finished deduplication our data requires some preprocessing before we go on further with analysis and making the prediction model.

Hence in the Preprocessing phase we do the following in the order below:-

1. Begin by removing the html tags
2. Remove any punctuations or limited set of special characters like , or . or # etc.
3. Check if the word is made up of english letters and is not alpha-numeric
4. Check to see if the length of the word is greater than 2 (as it was researched that there is no adjective in 2-letters)
5. Convert the word to lowercase
6. Remove Stopwords
7. Finally Snowball Stemming the word (it was obsereved to be better than Porter Stemming)<br>

After which we collect the words used to describe positive and negative reviews

In [63]:
# find sentences containing HTML tags
import re
i=0;
for sent in final['Text'].values:
    if (len(re.findall('<.*?>', sent))):
        print(i)
        print(sent)
        break;
    i += 1;    

        

8
What happens when you say his name three times? Michael Keaten stars in this comedy about two couples that live in an old two story house.  While coming back from a supply store, the couple suddenly get caught inside of a  &quot;broken-up&quot; bridge and then just before they start to tumble down  into the lake, a board catches them.  But just when they've got their hopes  up, and small dog steps on the board and the car starts to slide off the  bridge and into the lake waters.  A few minutes later...<p>They find  themselves back into their home, they find that somehow somehad light the  fireplace, as if done by magic.  From then on, they find a weird-looking  dead guy known as Bettlejuice.  The only way they can get him for help is  to call him by his name three times and he will appear at their survice.  But they soon wish that they have never called his name, because  Bettlejuice was once a troublemaker but he is the only one who can save  them, on the account that they said his 

In [4]:
import re
# Tutorial about Python regular expressions: https://pymotw.com/2/re/
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

stop = set(stopwords.words('english')) #set of stopwords
sno = nltk.stem.SnowballStemmer('english') #initialising the snowball stemmer

def cleanhtml(sentence): #function to clean the word of any html-tags
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', sentence)
    return cleantext
def cleanpunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return  cleaned
print(stop)
print('************************************')
print(sno.stem('tasty'))

{'same', 'few', 'm', "doesn't", 'hadn', 'what', 'having', "mustn't", "you'll", "that'll", 'ours', "you're", 'ma', 'd', 'her', 'did', 'here', 'only', 'she', 'up', 'you', 'weren', 'doing', 'it', 'have', 'after', 'during', 'of', 've', 'hasn', 'why', 'didn', 'yourselves', 'had', 'where', 'a', 're', 'our', 'theirs', "hasn't", "aren't", "mightn't", 'just', 'against', 'into', 'doesn', 'yourself', 'off', 'further', 'don', 'each', 'once', 'do', 'aren', "weren't", 'below', 'until', 'himself', 'with', 'whom', 'under', 'been', 'through', 'how', 'ain', 'both', "shouldn't", 'isn', "you've", 'all', "won't", "don't", 'any', 'i', 'such', 'itself', 'they', 'mustn', 'won', 'about', 'their', 'being', 'yours', 'are', 'has', "didn't", 'were', 'own', "wouldn't", 'hers', 'on', 'out', 'can', 'he', 'for', 'should', 'll', 'which', "it's", 'by', "isn't", "needn't", 'because', 'or', 't', 'myself', 'if', 'as', 'so', 'this', 'mightn', 'him', 'your', 'other', 'and', 'we', "you'd", "she's", 'couldn', "couldn't", 'agai

In [65]:
#Code for implementing step-by-step the checks mentioned in the pre-processing phase
# this code takes a while to run as it needs to run on 500k sentences.
i=0
str1=' '
final_string=[]
all_positive_words=[] # store words from +ve reviews here
all_negative_words=[] # store words from -ve reviews here.
s=''
for sent in final['Text'].values:
    filtered_sentence=[]
    #print(sent);
    sent=cleanhtml(sent) # remove HTMl tags
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if((cleaned_words.isalpha()) & (len(cleaned_words)>2)):    
                if(cleaned_words.lower() not in stop):
                    s=(sno.stem(cleaned_words.lower())).encode('utf8')
                    filtered_sentence.append(s)
                    if (final['Score'].values)[i] == 'positive': 
                        all_positive_words.append(s) #list of all words used to describe positive reviews
                    if(final['Score'].values)[i] == 'negative':
                        all_negative_words.append(s) #list of all words used to describe negative reviews reviews
                else:
                    continue
            else:
                continue 
    #print(filtered_sentence)
    str1 = b" ".join(filtered_sentence) #final string of cleaned words
    #print("***********************************************************************")
    
    final_string.append(str1)
    i+=1

In [66]:
final['CleanedText']=final_string #adding a column of CleanedText which displays the data after pre-processing of the review 

In [13]:
final.head(3) #below the processed review can be seen in the CleanedText Column 


# store final table into an SQlLite table for future.
conn = sqlite3.connect('final.sqlite')
c=conn.cursor()
conn.text_factory = str
final.to_sql('Reviews', conn, flavor=None, schema=None, if_exists='replace', index=True, index_label=None, chunksize=None, dtype=None)

## Separating data into Train ,Cross-validation,Test

In [5]:
con = sqlite3.connect('final.sqlite')
final = pd.read_sql_query('SELECT * FROM Reviews',con)
from sklearn.cross_validation import train_test_split
from sklearn import cross_validation
def values(x):
    if x == 'positive':
        return 1
    return 0
X = final['Text'].values
label = final['Score']
Y = label.map(values)

## Test data split ##
X_train, X_test, Y_train, Y_test = cross_validation.train_test_split(X, Y, test_size=0.3, random_state=0)
 


In [16]:
con = sqlite3.connect('final.sqlite')
final = pd.read_sql_query('SELECT * FROM Reviews',con)
from sklearn.cross_validation import train_test_split
from sklearn import cross_validation
def values(x):
    if x == 'positive':
        return 1
    return 0
X = final['Text'].values
label = final['Score']
Y = label.map(values)

## Test data split ##
X_1, X_test, Y_1, Y_test = cross_validation.train_test_split(X, Y, test_size=0.3, random_state=0)
X_train, X_cv, Y_train, Y_cv = cross_validation.train_test_split(X, Y, test_size=0.3, random_state=0)

# Random Forest and GBDT

## [7.2.2] Bag of Words (BoW)

In [4]:
count_vect = CountVectorizer()
final_counts = count_vect.fit_transform(X_train)
X_final = count_vect.transform(X_test)

In [29]:
x_final_cv = count_vect.transform(X_cv)

### Finding hyperparameters of random forest

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
l1_values = [{'n_estimators': [5,6,7,8,9,10,11,12,13,14,15,16,20,30,40,60],
              'max_depth':[1,2,3,4,5,6,7,8,9,10]
              }]
model = GridSearchCV(RandomForestClassifier(), l1_values, scoring = 'f1', cv=10)
model.fit(final_counts[0:20000], Y_train[0:20000])
print('best alpha: ',model.best_estimator_)

best alpha:  RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=5, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


### Finding hyperparameters for GBDT

In [5]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
l1_values = [{'learning_rate':[10**-4,10**-3,10**-2,10**-1]}]
model = GridSearchCV(GradientBoostingClassifier(), l1_values, scoring = 'f1', cv=10)
model.fit(final_counts[0:20000], Y_train[0:20000])
print('best alpha: ',model.best_estimator_)

best alpha:  GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)


### Random forest  on bow

In [12]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
NB = RandomForestClassifier(max_depth=10,n_estimators=5)
NB.fit(final_counts[0:70000],Y_train[0:70000])
pred = NB.predict(X_final[0:10000])
f1score = f1_score(Y_test[0:10000],pred,average='weighted')
precision = precision_score(Y_test[0:10000],pred,average='weighted')
recall = recall_score(Y_test[0:10000],pred,average='weighted')
con_matrix = confusion_matrix(Y_test[0:10000],pred)
acc = accuracy_score(Y_test[0:10000],pred)
print('accuracy ',acc*100)
print('f1_score',f1score)
print('precision',precision)
print('recall',recall)
print('confusion matrix',con_matrix)

accuracy  84.33
f1_score 0.7718025446916046
precision 0.7895628025605121
recall 0.8433
confusion matrix [[   1 1566]
 [   1 8432]]


### GBDT on Bow

In [6]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
NB = GradientBoostingClassifier(learning_rate=0.1)
NB.fit(final_counts[0:70000],Y_train[0:70000])
pred = NB.predict(X_final[0:10000])
f1score = f1_score(Y_test[0:10000],pred,average='weighted')
precision = precision_score(Y_test[0:10000],pred,average='weighted')
recall = recall_score(Y_test[0:10000],pred,average='weighted')
con_matrix = confusion_matrix(Y_test[0:10000],pred)
acc = accuracy_score(Y_test[0:10000],pred)
print('accuracy ',acc*100)
print('f1_score',f1score)
print('precision',precision)
print('recall',recall)
print('confusion matrix',con_matrix)

accuracy  87.57000000000001
f1_score 0.8443309161886092
precision 0.8756169627926421
recall 0.8757
confusion matrix [[ 378 1189]
 [  54 8379]]


## [7.2.4] Bi-Grams and n-Grams.

**Motivation**

Now that we have our list of words describing positive and negative reviews lets analyse them.<br>

We begin analysis by getting the frequency distribution of the words as shown below

In [ ]:
freq_dist_positive=nltk.FreqDist(all_positive_words)
freq_dist_negative=nltk.FreqDist(all_negative_words)
print("Most Common Positive Words : ",freq_dist_positive.most_common(20))
print("Most Common Negative Words : ",freq_dist_negative.most_common(20))

<b>Observation:-</b> From the above it can be seen that the most common positive and the negative words overlap for eg. 'like' could be used as 'not like' etc. <br>
So, it is a good idea to consider pairs of consequent words (bi-grams) or q sequnce of n consecutive words (n-grams)

In [7]:
#bi-gram, tri-gram and n-gram

#removing stop words like "not" should be avoided before building n-grams
count_vect = CountVectorizer(ngram_range=(1,2) ) #in scikit-learn
final_bigram_counts = count_vect.fit_transform(X_train)
X_test_grams = count_vect.transform(X_test)
#x_cv_grams = count_vect.transform(X_cv)

### Finding hyperparameters of random forest

In [15]:
l1_values = [{'n_estimators': [5,6,7,8,9,10,11,12,13,14,15,16,20,30,40,60],
              'max_depth':[1,2,3,4,5,6,7,8,9,10]
              }]
model = GridSearchCV(RandomForestClassifier(), l1_values, scoring = 'f1', cv=10)
model.fit(final_bigram_counts[0:5000], Y_train[0:5000])
print(model.best_estimator_)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=7, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=5, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


### Finding hyperparameters on GBDT

In [8]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
l1_values = [{'learning_rate':[10**-4,10**-3,10**-2,10**-1]}]
model = GridSearchCV(GradientBoostingClassifier(), l1_values, scoring = 'f1', cv=10)
model.fit(final_bigram_counts[0:5000], Y_train[0:5000])
print('best alpha: ',model.best_estimator_)

best alpha:  GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)


## Random Forest on bi-grams

In [19]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
NB = RandomForestClassifier(n_estimators=5,max_depth=7)
NB.fit(final_bigram_counts,Y_train)
pred = NB.predict(X_test_grams[0:10000])
f1score = f1_score(Y_test[0:10000],pred,average='weighted')
precision = precision_score(Y_test[0:10000],pred,average='weighted')
recall = recall_score(Y_test[0:10000],pred,average='weighted')
con_matrix = confusion_matrix(Y_test[0:10000],pred)
accuracy = accuracy_score(Y_test[0:10000],pred)
print('accuracy score',accuracy*100)
print('f1_score',f1score)
print('precision',precision)
print('recall',recall)
print('confusion matrix',con_matrix)

accuracy score 84.33
f1_score 0.7716105788531438
precision 0.7111548900000001
recall 0.8433
confusion matrix [[   0 1567]
 [   0 8433]]


E:\anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
E:\anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## GBDT on bi-grams

In [9]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
NB = GradientBoostingClassifier(learning_rate=0.1)
NB.fit(final_bigram_counts,Y_train)
pred = NB.predict(X_test_grams[0:10000])
f1score = f1_score(Y_test[0:10000],pred,average='weighted')
precision = precision_score(Y_test[0:10000],pred,average='weighted')
recall = recall_score(Y_test[0:10000],pred,average='weighted')
con_matrix = confusion_matrix(Y_test[0:10000],pred)
accuracy = accuracy_score(Y_test[0:10000],pred)
print('accuracy score',accuracy*100)
print('f1_score',f1score)
print('precision',precision)
print('recall',recall)
print('confusion matrix',con_matrix)

accuracy score 87.87
f1_score 0.8489667201075903
precision 0.8802056143898921
recall 0.8787
confusion matrix [[ 403 1164]
 [  49 8384]]


# [7.2.5] TF-IDF

In [6]:

tf_idf_vect = TfidfVectorizer(ngram_range=(1,2))
final_tf_idf = tf_idf_vect.fit_transform(X_train)
X_cv_tfidf = tf_idf_vect.transform(X_test)

### finding hyperparameters of random forest

In [21]:
l1_values = [{'n_estimators': [5,6,7,8,9,10,11,12,13,14,15,16,20,30,40,60],
              'max_depth':[1,2,3,4,5,6,7,8,9,10]
              }]
model = GridSearchCV(RandomForestClassifier(), l1_values, scoring = 'f1', cv=10)
model.fit(final_tf_idf[0:5000], Y_train[0:5000])
print(model.best_estimator_)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=8, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=5, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


In [7]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
l1_values = [{'learning_rate':[10**-4,10**-3,10**-2,10**-1]}]
model = GridSearchCV(GradientBoostingClassifier(), l1_values, scoring = 'f1', cv=10)
model.fit(final_tf_idf[0:5000], Y_train[0:5000])
print('best alpha: ',model.best_estimator_)

best alpha:  GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)


In [22]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
NB = RandomForestClassifier(max_depth=8,n_estimators=5)
NB.fit(final_tf_idf,Y_train)
pred = NB.predict(X_cv_tfidf[0:5000])
f1score = f1_score(Y_test[0:5000],pred,average='weighted')
precision = precision_score(Y_test[0:5000],pred,average='weighted')
recall = recall_score(Y_test[0:5000],pred,average='weighted')
con_matrix = confusion_matrix(Y_test[0:5000],pred)
accuracy = accuracy_score(Y_test[0:5000],pred)
print('accuracy score',accuracy*100)
print('f1_score',f1score)
print('precision',precision)
print('recall',recall)
print('confusion matrix',con_matrix)

accuracy score 84.61999999999999
f1_score 0.7757062506770663
precision 0.71605444
recall 0.8462
confusion matrix [[   0  769]
 [   0 4231]]


E:\anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
E:\anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [12]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
NB = GradientBoostingClassifier(learning_rate=0.1)
NB.fit(final_tf_idf,Y_train)
pred = NB.predict(X_cv_tfidf[0:5000])
f1score = f1_score(Y_test[0:5000],pred,average='weighted')
precision = precision_score(Y_test[0:5000],pred,average='weighted')
recall = recall_score(Y_test[0:5000],pred,average='weighted')
con_matrix = confusion_matrix(Y_test[0:5000],pred)
accuracy = accuracy_score(Y_test[0:5000],pred)
print('accuracy score',accuracy*100)
print('f1_score',f1score)
print('precision',precision)
print('recall',recall)
print('confusion matrix',con_matrix)

accuracy score 88.46000000000001
f1_score 0.8577438320161369
precision 0.8863114705882353
recall 0.8846
confusion matrix [[ 216  553]
 [  24 4207]]


In [ ]:
 # source: https://buhrmann.github.io/tfidf-analysis.html
def top_tfidf_feats(row, features, top_n=25):
    ''' Get top n tfidf values in row and return them with their corresponding feature names.'''
    topn_ids = np.argsort(row)[::-1][:top_n]
    top_feats = [(features[i], row[i]) for i in topn_ids]
    df = pd.DataFrame(top_feats)
    df.columns = ['feature', 'tfidf']
    return df

top_tfidf = top_tfidf_feats(final_tf_idf[1,:].toarray()[0],features,25)

In [ ]:
top_tfidf


# [7.2.6] Word2Vec

In [ ]:
# Using Google News Word2Vectors
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

# in this project we are using a pretrained model by google
# its 3.3G file, once you load this into your memory 
# it occupies ~9Gb, so please do this step only if you have >12G of ram
# we will provide a pickle file wich contains a dict , 
# and it contains all our courpus words as keys and  model[word] as values
# To use this code-snippet, download "GoogleNews-vectors-negative300.bin" 
# from https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit
# it's 1.9GB in size.


model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)



In [13]:
import gensim
i=0
list_of_sent=[]
for sent in X_train:
    filtered_sentence=[]
    sent=cleanhtml(sent)
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if(cleaned_words.isalpha()):    
                filtered_sentence.append(cleaned_words.lower())
            else:
                continue 
    list_of_sent.append(filtered_sentence)

E:\anaconda\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [24]:
import gensim
i=0
list_of_sent_xcv=[]
for sent in X_cv:
    filtered_sentence=[]
    sent=cleanhtml(sent)
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if(cleaned_words.isalpha()):    
                filtered_sentence.append(cleaned_words.lower())
            else:
                continue 
    list_of_sent_xcv.append(filtered_sentence)

In [14]:
i =0
list_of_Xtest =[]
for sent in X_test:
    filtered_Xtest = []
    sent = cleanhtml(sent)
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if(cleaned_words.isalpha()):
                filtered_Xtest.append(cleaned_words.lower())
            else:
                continue
    list_of_Xtest.append(filtered_Xtest)

In [15]:
w2v_model=gensim.models.Word2Vec(list_of_sent,min_count=5,size=50, workers=4)    


## Tf-IDF Word2vec Avg W2V

In [16]:

sent_vectors = []; # the avg-w2v for each sentence/review is stored in this list
for sent in list_of_sent: # for each review/sentence
    sent_vec = np.zeros(50) # as word vectors are of zero length
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        try:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
        except:
            pass
    sent_vec /= cnt_words
    sent_vectors.append(sent_vec)
print(len(sent_vectors))
print(len(sent_vectors[0]))

E:\anaconda\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in true_divide
  del sys.path[0]


254919
50


In [17]:
sent_vectors_xtest = []; # the avg-w2v for each sentence/review is stored in this list
for sent in list_of_Xtest: # for each review/sentence
    sent_vec = np.zeros(50) # as word vectors are of zero length
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        try:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
        except:
            pass
    sent_vec /= cnt_words
    sent_vectors_xtest.append(sent_vec)

In [29]:
sent_vectors_xcv = []; # the avg-w2v for each sentence/review is stored in this list
for sent in list_of_sent_xcv: # for each review/sentence
    sent_vec = np.zeros(50) # as word vectors are of zero length
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        try:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
        except:
            pass
    sent_vec /= cnt_words
    sent_vectors_xcv.append(sent_vec)

### Finding the right depth of tree uing grid search

In [31]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
l1_values = [{'n_estimators': [5,6,7,8,9,10,11,12,13,14,15,16,20,30,40,60],
              'max_depth':[1,2,3,4,5,6,7,8,9,10]
              }]
model = GridSearchCV(RandomForestClassifier(), l1_values, scoring = 'f1', cv=10)
model.fit(sent_vectors[0:20000], Y_train[0:20000])
print('best alpha: ',model.best_estimator_)

best alpha:  RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=60, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


In [32]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
dt = RandomForestClassifier(max_depth=10,n_estimators=60)
dt.fit(sent_vectors[0:60000],Y_train[0:60000])
pred = dt.predict(sent_vectors_xtest[0:10000])
acc = accuracy_score(Y_test[0:10000],pred)
f1score = f1_score(Y_test[0:10000],pred,average='weighted')
precision = precision_score(Y_test[0:10000],pred,average='weighted')
recall = recall_score(Y_test[0:10000],pred,average='weighted')
con_matrix = confusion_matrix(Y_test[0:10000],pred)
print('accuracy ',acc*100)
print('f1_score',f1score)
print('precision',precision)
print('recall',recall)
print('confusion matrix',con_matrix)
print('decision tree')


accuracy  86.1
f1_score 0.8193355444738
precision 0.8510917319637511
recall 0.861
confusion matrix [[ 244 1323]
 [  67 8366]]
decision tree


In [18]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
dt = GradientBoostingClassifier(learning_rate=0.1,n_estimators=60)
dt.fit(sent_vectors[0:60000],Y_train[0:60000])
pred = dt.predict(sent_vectors_xtest[0:10000])
acc = accuracy_score(Y_test[0:10000],pred)
f1score = f1_score(Y_test[0:10000],pred,average='weighted')
precision = precision_score(Y_test[0:10000],pred,average='weighted')
recall = recall_score(Y_test[0:10000],pred,average='weighted')
con_matrix = confusion_matrix(Y_test[0:10000],pred)
print('accuracy ',acc*100)
print('f1_score',f1score)
print('precision',precision)
print('recall',recall)
print('confusion matrix',con_matrix)
print('decision tree')

accuracy  86.8
f1_score 0.8442198279269145
precision 0.8505088450389289
recall 0.868
confusion matrix [[ 450 1117]
 [ 203 8230]]
decision tree


## Decision tree on Tf-IDF Word2vec

In [33]:
# TF-IDF weighted Word2Vec

tfidf_feat = tf_idf_vect.get_feature_names() # tfidf words/col-names
# final_tf_idf is the sparse matrix with row= sentence, col=word and cell_val = tfidf

tfidf_sent_vectors = []; # the tfidf-w2v for each sentence/review is stored in this list
row=0;
for sent in list_of_sent[0:2000]: # for each review/sentence
    sent_vec = np.zeros(50) # as word vectors are of zero length
    weight_sum =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        try:
            vec = w2v_model.wv[word]
            # obtain the tf_idfidf of a word in a sentence/review
            tfidf = final_tf_idf[row, tfidf_feat.index(word)]
            sent_vec += (vec * tfidf)
            weight_sum += tfidf
        except:
            pass
    sent_vec /= weight_sum
    tfidf_sent_vectors.append(sent_vec)
    row += 1
    

    

In [34]:
test_tfidf = tf_idf_vect.fit_transform(X_test)

In [36]:
tfidf_feat = tf_idf_vect.get_feature_names() # tfidf words/col-names
# final_tf_idf is the sparse matrix with row= sentence, col=word and cell_val = tfidf

tfidf_sent_vectors_xtest = []; # the tfidf-w2v for each sentence/review is stored in this list
row=0;
for sent in list_of_Xtest[0:100]: # for each review/sentence
    sent_vec = np.zeros(50) # as word vectors are of zero length
    weight_sum =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        try:
            vec = w2v_model.wv[word]
            # obtain the tf_idfidf of a word in a sentence/review
            tfidf = test_tfidf[row, tfidf_feat.index(word)]
            sent_vec += (vec * tfidf)
            weight_sum += tfidf
        except:
            pass
    sent_vec /= weight_sum
    tfidf_sent_vectors_xtest.append(sent_vec)
    row += 1
    

In [38]:
l1_values = [{'learning_rate': [5,6,7,8,9,10,11,12,13,14,15,16,20,30,40,60],
              'max_depth':[1,2,3,4,5,6,7,8,9,10]
              }]
model = GridSearchCV(RandomForestClassifier(), l1_values, scoring = 'f1', cv=10)
model.fit(tfidf_sent_vectors[0:2000], Y_train[0:2000])
print('best alpha: ',model.best_estimator_)

best alpha:  RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=9, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=13, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


In [41]:
l1_values = [{'learning_rate': [10**-4,10**-3,10**-2,10**-1]}]
model = GridSearchCV(GradientBoostingClassifier(), l1_values, scoring = 'f1', cv=10)
model.fit(tfidf_sent_vectors[0:2000], Y_train[0:2000])
print('best alpha: ',model.best_estimator_)

best alpha:  GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)


In [40]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
NB = GradientBoostingClassifier(learning_rate=0.1)
NB.fit(tfidf_sent_vectors[0:2000],Y_train[0:2000])
pred = NB.predict(tfidf_sent_vectors_xtest[0:100])
acc = accuracy_score(Y_test[0:100],pred)
f1score = f1_score(Y_test[0:100],pred,average='weighted')
precision = precision_score(Y_test[0:100],pred,average='weighted')
recall = recall_score(Y_test[0:100],pred,average='weighted')
con_matrix = confusion_matrix(Y_test[0:100],pred)
print('accuracy ',acc*100)
print('f1_score',f1score)
print('precision',precision)
print('recall',recall)
print('confusion matrix',con_matrix)

accuracy  85.0
f1_score 0.8216280925778132
precision 0.8164893617021276
recall 0.85
confusion matrix [[ 3 12]
 [ 3 82]]


In [39]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
NB = RandomForestClassifier(max_depth=1,n_estimators=13)
NB.fit(tfidf_sent_vectors[0:2000],Y_train[0:2000])
pred = NB.predict(tfidf_sent_vectors_xtest[0:100])
acc = accuracy_score(Y_test[0:100],pred)
f1score = f1_score(Y_test[0:100],pred,average='weighted')
precision = precision_score(Y_test[0:100],pred,average='weighted')
recall = recall_score(Y_test[0:100],pred,average='weighted')
con_matrix = confusion_matrix(Y_test[0:100],pred)
print('accuracy ',acc*100)
print('f1_score',f1score)
print('precision',precision)
print('recall',recall)
print('confusion matrix',con_matrix)

accuracy  85.0
f1_score 0.781081081081081
precision 0.7225
recall 0.85
confusion matrix [[ 0 15]
 [ 0 85]]


E:\anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
E:\anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## Conclusion and summary

#### Performance table of Random Forest

|      Algorithms                       | Bow       | Bigrams  | tf_idf  | avgword2vec   | tfidf_word2vec  |
|---------------------------------------|-----------|----------|---------|---------------|-----------------|
|        max_depth                      |    10     |     7    |    8    |      10       |        9        |
|---------------------------------------|-----------|----------|---------|---------------|-----------------|
|        n_estimators                   |    5      |     5    |    5    |      60       |        13       |
|---------------------------------------|-----------|----------|---------|---------------|-----------------|
|       precision                       |  0.789    |   0.771  |  0.716  |    0.851      |      0.816      |
|---------------------------------------|-----------|----------|---------|---------------|-----------------|
|       recall                          |  0.843    |   0.843  |  0.846  |    0.861      |      0.850      |
|---------------------------------------|-----------|----------|---------|---------------|-----------------|
|       f1_score                        |  0.771    |   0.777  |  0.775  |    0.819      |      0.821      |
|---------------------------------------|-----------|----------|---------|---------------|-----------------|
|       accuracy                        |   84.33   |   84.33  |  84.61  |    86.12      |       85.0      |
|---------------------------------------|-----------|----------|---------|---------------|-----------------|


#### Performance table for GBDT

|      Algorithms                       | Bow       | Bigrams  | tf_idf  | avgword2vec   | tfidf_word2vec  |
|---------------------------------------|-----------|----------|---------|---------------|-----------------|
|        learning_rate                  |  0.1      |    0.1   |   0.1   |     0.1       |       0.1       |
|---------------------------------------|-----------|----------|---------|---------------|-----------------|
|       precision                       |  0.875    |   0.880  |  0.886  |    0.850      |      0.722      |
|---------------------------------------|-----------|----------|---------|---------------|-----------------|
|       recall                          |  0.877    |   0.878  |  0.884  |    0.868      |      0.850      |
|---------------------------------------|-----------|----------|---------|---------------|-----------------|
|       f1_score                        |  0.844    |   0.848  |  0.857  |    0.844      |      0.781      |
|---------------------------------------|-----------|----------|---------|---------------|-----------------|
|       accuracy                        |   87.57   |   87.87  |  88.46  |    86.80      |       85.1      |
|---------------------------------------|-----------|----------|---------|---------------|-----------------|


### Confusion matrix of Random forest

#### Bow

|  1 |1566|
|----|----|
|1   |8432|

#### Bigrams

|0    |1567 |
|-----|-----|
| 0   |8423 |

#### tf_idf

| 0  | 769|
|----|----|
| 0  |4231|

#### Avg word2vec

|  244  | 1323  |
|-------|-------|
|  64   | 8366  |

#### tf_idf weighted word2vec

|   3    |   12   |
|--------|--------|
|   3    |   82   |

### Confusion matrix of GBDT

#### bow

|378 |1189|
|----|----|
|54  |8379|

#### bi-grams

|403 |1164|
|----|----|
|49  |8384|

#### tf-idf

|216 |553 |
|----|----|
|24  |4207|

#### Avg word2vec

| 450|1117|
|----|----|
|203 |8230|

#### tf-idf weighted word2vec

|  0 |15  |
|----|----|
|0   |85  |